In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import pandas as pd

import tensorflow as tf

from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split


from input_generator import load_raw_data, get_tags_as_inputs
from data_parser import get_tags_and_labels, get_vocab, get_tags

d:\miniconda3\envs\deep\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
d:\miniconda3\envs\deep\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
d:\miniconda3\envs\deep\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
d:\miniconda3\envs\deep\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (

In [2]:
csvpath = 'data/CAvideos.cvs'
tab_dataframe = get_tags_and_labels(csvpath)

# tab_dataframe.dropna(inplace = True) 
# tab_dataframe["tags"]= tab_dataframe["tags"].str.split('.')

# Creating different coloumn based on different tags

new = tab_dataframe["tags"].str.split(".", expand = True)

for i in range(0, 124):
    name = "tags"+str(i)
    tab_dataframe[name] = new[i]

# Dropping old Name columns 
tab_dataframe.drop(columns =["tags"], inplace = True) 
tab_dataframe.fillna("NA", inplace = True) 


tab_dataframe.head()

,category_id,tags0,tags1,tags2,tags3,tags4,tags5,tags6,tags7,tags8,...,tags114,tags115,tags116,tags117,tags118,tags119,tags120,tags121,tags122,tags123
0,10,Eminem,Walk,On,Water,Aftermath/Shady/Interscope,Rap,NA,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
1,23,plush,bad unboxing,unboxing,fan mail,idubbbztv,idubbbztv2,things,best,packages,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
2,23,racist superman,rudy,mancuso,king,bach,racist,superman,love,rudy mancuso poo bear black white official mus...,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
3,24,ryan,higa,higatv,nigahiga,i dare you,idy,rhpc,dares,no truth,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
4,10,edsheeran,ed sheeran,acoustic,live,cover,official,remix,official video,lyrics,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


In [3]:
vocab = get_vocab(get_tags(csvpath))
voc_di = {vo:i for i,vo in enumerate(vocab)}

In [4]:
for index in range(0, 124):
    tab_dataframe['tags{}'.format(index)] = tab_dataframe['tags{}'.format(index)].map(voc_di)
    
tab_dataframe

,category_id,tags0,tags1,tags2,tags3,tags4,tags5,tags6,tags7,tags8,...,tags114,tags115,tags116,tags117,tags118,tags119,tags120,tags121,tags122,tags123
0,10,32396.0,38062.0,35721.0,123655.0,3115.0,733.0,32162.0,32162.0,32162.0,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
1,23,132103.0,29540.0,1140.0,81646.0,34528.0,98107.0,143090.0,32968.0,11245.0,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
2,23,3721.0,82149.0,7632.0,40842.0,77052.0,118834.0,79019.0,144150.0,143145.0,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
3,24,89704.0,121504.0,98142.0,80969.0,3722.0,76815.0,513.0,128632.0,94935.0,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
4,10,71696.0,120630.0,40097.0,132359.0,17139.0,81070.0,99809.0,138576.0,143233.0,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
5,25,NaN,143152.0,38342.0,61250.0,124798.0,142036.0,19168.0,115530.0,61008.0,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
6,23,139703.0,132456.0,82216.0,25252.0,26367.0,15938.0,88766.0,68373.0,856.0,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
7,22,NaN,32162.0,32162.0,32162.0,32162.0,32162.0,32162.0,32162.0,32162.0,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
8,24,14039.0,18084.0,72859.0,109881.0,67732.0,82341.0,6308.0,8844.0,42317.0,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
9,22,40196.0,101724.0,142836.0,32162.0,32162.0,32162.0,32162.0,32162.0,32162.0,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
